#**Pre-request**

##Mount google drive


In [2]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install pakages


In [3]:
#Install pakages
%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/clustring_requirements.txt --no-cache-dir



In [4]:
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
clustring_requirements-lock.txt  requirements-lock.txt
clustring_requirements.txt       requirements.txt
configs/                         results/
dataset/                         run_experiment.py
Extract_requirements-lock.txt    sample_extract_requirements-lock.txt
Extract_requirements.txt         src/
notebooks/                       tests/
README.md


##Import  libs

In [5]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import logging
import yaml
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import altair as alt
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
import altair as alt
import numpy as np
import pandas as pd
import altair as alt

# Try the best transformer for large datasets
try:
    alt.data_transformers.enable("vegafusion")
    print("VegaFusion enabled for large datasets.")
except Exception:
    print("VegaFusion not available — falling back to disable max rows limit.")
    alt.data_transformers.disable_max_rows()

logger = logging.getLogger(__name__)





%pip freeze > clustring_requirements-lock.txt



VegaFusion enabled for large datasets.


#Basic Methods

##Loging

In [6]:

# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/clustring_extract.log")
    ]
)
logger = logging.getLogger(__name__)



##Config

In [7]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file"""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"Loaded config from {config_path}")
    return config

## CDR dataset

In [8]:
def load_cdr(file_path, nrows=None):
    """Load a CSV file and safely parse datetime columns."""
    logger.info(f"Loading file: {file_path} (nrows={nrows})")
    df = pd.read_csv(file_path, nrows=nrows)

    # Auto-detect and parse datetime columns
    for col in df.columns:
        if "datetime" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # Optional: strip extra spaces in headers
    df.columns = df.columns.str.strip()
    return df



def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["dataset"]["base_path"]
    files = config["dataset"]["files"]
    sample_size = config.get("training", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
        logger.info(f"Loaded {name} -> {df.shape} from {path}")
    return data


#Clustring

##  t-SNE Helper Function

In [9]:
def run_tsne_per_dataset(data_raw):
    print("\n============================\n t-SNE per Dataset \n============================")
    for name, df in data_raw.items():
        print(f"\n📄 Running t-SNE for {name.upper()} dataset")

        # skip user merge if not needed
        if "label" not in df.columns:
            print(f"⚠️ Skipping {name}: no label column.")
            continue

        numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
        numeric_cols = [c for c in numeric_cols if c not in ["label"]]
        if len(numeric_cols) < 2:
            print(f"⚠️ Skipping {name}: only {len(numeric_cols)} numeric feature(s).")
            continue

        df = df.dropna(subset=["label"])
        df[numeric_cols] = df[numeric_cols].fillna(0)

        scaler = StandardScaler()
        scaled_feats = scaler.fit_transform(df[numeric_cols])
        tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200)
        emb = tsne.fit_transform(scaled_feats)
        df["tsne_0"], df["tsne_1"] = emb[:, 0], emb[:, 1]

        chart = (
            alt.Chart(df, title=f"{name.upper()} Dataset t-SNE")
            .mark_circle(size=60, opacity=0.7)
            .encode(
                x=alt.X("tsne_0", title="t-SNE Dimension 1"),
                y=alt.Y("tsne_1", title="t-SNE Dimension 2"),
                color=alt.Color("label:N", title="Fraud Label", scale=alt.Scale(scheme="dark2")),
                tooltip=["phone_no_m", "label"]
            )
            .properties(width=600, height=500)
            .interactive()
        )
        display(chart)


##Distribution analysis

In [10]:
def plot_fraud_distribution(
    data_raw_labeled,
    dataset_name,
    feature_col,
    title=None,
    top_n=None,
    min_users=3,
    size_field="fraud_users",
    color_field="fraud_ratio"
):
    """
    Generic fraud distribution analyzer for any categorical feature.

    Parameters
    ----------
    data_raw_labeled : dict[str, pd.DataFrame]
        Dictionary of labeled raw datasets (e.g., {"app": df_app, "voc": df_voc, ...})
    dataset_name : str
        Which dataset to use ("app", "voc", "sms", "user", etc.)
    feature_col : str
        The categorical column to group by (e.g., "busi_name", "calltype_id", "city_name")
    title : str, optional
        Custom chart title (default auto-generated)
    top_n : int or None
        Show only the top N by `size_field`. If None, show all.
    min_users : int
        Minimum unique users required to include a category (default 3)
    size_field : str
        Column to use for bubble size ("fraud_users", "users", or "fraud_ratio")
    color_field : str
        Column to use for bubble color ("fraud_ratio", "fraud_users", or "users")
    """

    import pandas as pd
    import altair as alt

    # --- Validation ---
    if dataset_name not in data_raw_labeled:
        print(f"❌ Dataset '{dataset_name}' not found.")
        return

    df = data_raw_labeled[dataset_name].copy()
    required_cols = {"phone_no_m", "label", feature_col}
    missing = required_cols - set(df.columns)
    if missing:
        print(f"❌ Missing columns in {dataset_name}: {missing}")
        return

    # --- Aggregate unique users and unique fraud users ---
    summary = (
        df.groupby(feature_col)
        .agg(
            users=("phone_no_m", "nunique"),
            fraud_users=("phone_no_m", lambda x: x[df.loc[x.index, "label"] == 1].nunique())
        )
        .reset_index()
    )

    summary["fraud_ratio"] = summary["fraud_users"] / summary["users"]
    summary = summary.query("users >= @min_users").fillna(0)

    # --- Sort and clip ---
    if size_field not in summary.columns:
        print(f"⚠️ '{size_field}' not found. Defaulting to 'fraud_users'.")
        size_field = "fraud_users"

    if color_field not in summary.columns:
        print(f"⚠️ '{color_field}' not found. Defaulting to 'fraud_ratio'.")
        color_field = "fraud_ratio"

    summary_sorted = summary.sort_values(size_field, ascending=False)
    summary_plot = summary_sorted.head(top_n) if top_n else summary_sorted

    # --- Title ---
    chart_title = title or f"Fraud Distribution by {feature_col} ({dataset_name.upper()})"

    # --- Visualization ---
    chart = (
        alt.Chart(summary_plot, title=chart_title)
        .mark_circle(opacity=0.8)
        .encode(
            x=alt.X("users:Q", title="Number of Users (log)", scale=alt.Scale(type="log")),
            y=alt.Y("fraud_ratio:Q", title="Fraud Ratio"),
            size=alt.Size(f"{size_field}:Q", title=size_field.replace("_", " ").title(),
                          scale=alt.Scale(range=[10, 800])),
            color=alt.Color(f"{color_field}:Q", title=color_field.replace("_", " ").title(),
                            scale=alt.Scale(scheme="reds")),
            tooltip=[feature_col, "users", "fraud_users", "fraud_ratio"]
        )
        .properties(width=650, height=500)
        .interactive()
    )

    display(chart)

    # --- Table Output ---
    print(f"\nTop {'All' if not top_n else top_n} '{feature_col}' by {size_field} in {dataset_name.upper()}:")
    display(summary_plot)

    return summary_plot


## ⚙️ Load Config and Data

##RAW

###T-SNE

In [11]:
# =====================================================
# 📂 Load RAW Datasets
# =====================================================
config = load_config("configs/baseline.yaml")

raw_cfg = config["clustring"]["raw"]
raw_base = raw_cfg["base_path"]
raw_files = {k: os.path.join(raw_base, v) for k, v in raw_cfg["files"].items()}

# --- Verify existence
for name, path in raw_files.items():
    if not os.path.exists(path):
        print(f"⚠️ Warning: RAW file not found for {name}: {path}")

# --- Load
data_raw = {k: pd.read_csv(p) for k, p in raw_files.items()}

print("\n✅ Loaded RAW datasets:")
for name, df in data_raw.items():
    print(f"RAW {name}: {df.shape}")

# =====================================================
# 🏷️ Merge labels once for all non-user datasets
# =====================================================
user_labels = data_raw["user"][["phone_no_m", "label"]]

data_raw_labeled = {}
for name, df in data_raw.items():
    if name == "user":
        data_raw_labeled[name] = df
        continue
    merged_df = df.merge(user_labels, on="phone_no_m", how="left")
    data_raw_labeled[name] = merged_df

print("\n✅ Merged fraud labels into all datasets.")



✅ Loaded RAW datasets:
RAW app: (26142, 4)
RAW sms: (64225, 4)
RAW user: (60, 13)
RAW voc: (48190, 8)

✅ Merged fraud labels into all datasets.


In [12]:



# =====================================================
# 🧠 Run standard t-SNE per dataset (fraud vs legit)
# =====================================================
print("\n============================\n t-SNE per Dataset \n============================")
#run_tsne_per_dataset(data_raw_labeled)



 t-SNE per Dataset 


###Distribution

In [15]:
plot_fraud_distribution(data_raw_labeled, "app", "busi_name", size_field="users", color_field="fraud_ratio", title="Top 20 Apps by Fraud Volume and Risk")
plot_fraud_distribution(data_raw_labeled, "sms", "calltype_id", size_field="users", color_field="fraud_ratio", title="Top 20 SMS  type by Fraud Volume and Risk")
plot_fraud_distribution(data_raw_labeled, "voc", "calltype_id", size_field="users", color_field="fraud_ratio", title="Top 20 Voice call type by Fraud Volume and Risk")
plot_fraud_distribution(data_raw_labeled, "voc", "city_name", size_field="users", color_field="fraud_ratio", title="Top 20 voice city name by Fraud Volume and Risk")


alt.Chart(...)


Top All 'busi_name' by users in APP:


,busi_name,users,fraud_users,fraud_ratio
197,DNS,49,22,0.448980
1189,腾讯网,41,17,0.414634
1159,网页浏览,41,14,0.341463
930,微信,40,15,0.375000
262,HTTPS,37,13,0.351351
...,...,...,...,...
1229,词典翻译金山词霸,3,2,0.666667
43,51YES流量统计,3,0,0.000000
67,AD调查,3,1,0.333333
68,AIM,3,1,0.333333


alt.Chart(...)


Top All 'calltype_id' by users in SMS:


...

,calltype_id,users,fraud_users,fraud_ratio
0,1,59,29,0.491525
1,2,41,15,0.365854
2,3,3,1,0.333333


alt.Chart(...)


Top All 'city_name' by users in VOC:


,city_name,users,fraud_users,fraud_ratio
6,成都,36,16,0.444444
5,德阳,7,3,0.428571
12,资阳,7,1,0.142857
10,绵阳,7,3,0.428571
15,雅安,5,1,0.200000
9,眉山,5,1,0.200000
1,凉山,4,0,0.000000
4,广元,4,1,0.250000
13,达州,4,2,0.500000
8,甘孜,3,1,0.333333


,city_name,users,fraud_users,fraud_ratio
6,成都,36,16,0.444444
5,德阳,7,3,0.428571
12,资阳,7,1,0.142857
10,绵阳,7,3,0.428571
15,雅安,5,1,0.200000
9,眉山,5,1,0.200000
1,凉山,4,0,0.000000
4,广元,4,1,0.250000
13,达州,4,2,0.500000
8,甘孜,3,1,0.333333


##Agg